In [1]:
import os
import pandas as pd
import numpy as np
import pyreadstat


Process of Colorectal_161
   - characteristic.csv process

In [79]:
import pandas as pd

df_char = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_161/characteristic.csv')

df_char = df_char.rename(columns={
    'ADHERENC': 'adherence',
    'agecat': 'age',
    'ARM': 'arm',
    'BAD_TOX': 'serious adverse effect',
    'BWL_OBS': 'bowel obstruction',
    'BWL_PERF': 'bowel perforation',
    'HISTO_G': 'histology',
    'PS': 'ECOG performance status',
    'racecat': 'race',
    'SEX':'sex',
    'NUMCYCLE': 'number of chemotherapy cycles received',
    'wild':'biomarker KRAS',
    'bmi2':'bmi',
}
)

df_char['age'] = df_char['age'].replace({
    '< 40': 40,
    '40-69': 55,
    '>= 70': 70,
}).astype(float)
df_char['race']= df_char['race'].replace({'oth': 'other', 'w': 'white', 'b': 'black'})
df_char['adherence'] = df_char['adherence'].apply(lambda x: 'Yes' if x == 1 else 'No')
df_char['bowel obstruction'] = df_char['bowel obstruction'].apply(lambda x: 'Yes' if x == 1 else 'No')
df_char['bowel perforation'] = df_char['bowel perforation'].apply(lambda x: 'Yes' if x == 1 else 'No')
df_char['serious adverse effect'] = df_char['serious adverse effect'].apply(lambda x: 'Yes' if x == 1 else 'No')
df_char['histology'] = df_char['histology'].replace({
    1:'poorly differentiated',
    2:'well differentiated',
})
df_char['sex'] = df_char['sex'].replace({'m':'male','f':'female'})
arm_mapping = {
    'A': 'FOLFOX',
    'B': 'FOLFIRI',
    'C': 'FOLFIRI (KRAS mutant)',
    'D': 'FOLFOX + Cetuximab',
    'E': 'FOLFIRI + Cetuximab',
    'F': 'FOLFIRI + Cetuximab'
}
df_char['arm'] = df_char['arm'].map(arm_mapping)
df_char['biomarker KRAS'] = df_char['biomarker KRAS'].replace({0:'mutant',1:'wild-type'}).fillna('interminate')
df_char['bmi'] = df_char['bmi'].fillna(df_char['bmi'].median()).round(3)
df_char['number of chemotherapy cycles received'] = df_char['number of chemotherapy cycles received'].fillna(df_char['number of chemotherapy cycles received'].median()).round()
ecog_mappings = {
    0: 'fully active',
    1: 'symptoms but ambulatory',
    2: 'in bed less than 50% of the time'
}
freq_ecog = df_char['ECOG performance status'].map(ecog_mappings).mode()[0]
df_char['ECOG performance status'] = df_char['ECOG performance status'].map(ecog_mappings)
df_char['ECOG performance status'].fillna(value=freq_ecog, inplace=True)
df_char['ECOG performance status'].replace(to_replace=np.nan, value=freq_ecog, inplace=True)

df_CHAR = df_char[[
    'mask_id',
    'race',
    'sex',
    'age',
    'bmi',
    'arm',
    'adherence',
    'serious adverse effect',
    'bowel obstruction',
    'bowel perforation',
    'number of chemotherapy cycles received',
    'histology',
    'ECOG performance status',
    'biomarker KRAS'
]]

# print(df_CHAR.isnull().sum())
df_CHAR.head()

,mask_id,race,sex,age,bmi,arm,adherence,serious adverse effect,bowel obstruction,bowel perforation,number of chemotherapy cycles received,histology,ECOG performance status,biomarker KRAS
0,1,other,female,55.0,20.974,FOLFOX,No,No,No,No,12.0,well differentiated,fully active,wild-type
1,2,white,male,55.0,29.137,FOLFOX + Cetuximab,No,No,No,No,12.0,well differentiated,fully active,wild-type
2,3,white,female,55.0,49.271,FOLFOX + Cetuximab,No,No,No,No,12.0,well differentiated,fully active,wild-type
3,4,white,male,55.0,31.573,FOLFOX,No,No,No,No,12.0,well differentiated,fully active,mutant
4,5,white,male,55.0,16.777,FOLFOX + Cetuximab,Yes,No,No,No,12.0,poorly differentiated,fully active,wild-type


- object.csv and tox.csv process

In [80]:
df_obj = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_161/objectives.csv')
df_tox = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_161/tox.csv')

df_obj['target_label'] = df_obj['fustat8']

df_tox['is serious'] = df_tox['GRADE'].apply(lambda x: 1 if x > 2 else 0)
ae_list = df_tox['tox'].value_counts().index.tolist()
ae_list
ae_name_list = []
for ae in ae_list:
    ae_name = 'adverse effect: ' + ae.lower()
    df_tox[ae_name] = np.zeros(len(df_tox))
    df_tox.loc[df_tox['tox']==ae,ae_name] = 1
    df_tox[ae_name] = df_tox[ae_name] * df_tox['is serious']
    ae_name_list.append(ae_name)

df_tox = df_tox[ae_name_list+['mask_id']].groupby('mask_id').max().reset_index()
# df_tox.sum()
sub_ae_cols = ['adverse effect: thrombosis','adverse effect: hypersensitivity','adverse effect: diarrhea','adverse effect: infection','adverse effect: infarction']
df_chartox = pd.merge(df_CHAR, df_tox[sub_ae_cols+['mask_id']], how='left').fillna(0)
df_chartox[sub_ae_cols]= df_chartox[sub_ae_cols].replace({0:'No',1:'Yes'})
df_processed_161 = pd.merge(df_chartox, df_obj[['target_label','mask_id']])

df_processed_161.drop('mask_id', axis=1).to_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_161/Colorectal_161.csv')

num_feat_list = ['age', 'bmi', 'number of chemotherapy cycles received']
bin_feat_list = ['adherence', 'serious adverse effect', 'bowel obstruction', 'bowel perforation', 'adverse effect: thrombosis', 'adverse effect: hypersensitivity', 'adverse effect: diarrhea', 'adverse effect: infection', 'adverse effect: infarction']

with open('/home/alexandra/Downloads/datasets/ae_again/Colorectal_161/numerical_feature.txt','w') as f:
    for x in num_feat_list:
        f.write(x+'\n')
with open('/home/alexandra/Downloads/datasets/ae_again/Colorectal_161/binary_feature.txt','w') as f:
    for x in bin_feat_list:
        f.write(x+'\n')

df_processed_161.head()

,mask_id,race,sex,age,bmi,arm,adherence,serious adverse effect,bowel obstruction,bowel perforation,number of chemotherapy cycles received,histology,ECOG performance status,biomarker KRAS,adverse effect: thrombosis,adverse effect: hypersensitivity,adverse effect: diarrhea,adverse effect: infection,adverse effect: infarction,target_label
0,1,other,female,55.0,20.974,FOLFOX,No,No,No,No,12.0,well differentiated,fully active,wild-type,No,No,No,No,No,0
1,2,white,male,55.0,29.137,FOLFOX + Cetuximab,No,No,No,No,12.0,well differentiated,fully active,wild-type,No,No,No,No,No,0
2,3,white,female,55.0,49.271,FOLFOX + Cetuximab,No,No,No,No,12.0,well differentiated,fully active,wild-type,No,No,No,No,No,0
3,4,white,male,55.0,31.573,FOLFOX,No,No,No,No,12.0,well differentiated,fully active,mutant,No,No,No,No,No,0
4,5,white,male,55.0,16.777,FOLFOX + Cetuximab,Yes,No,No,No,12.0,poorly differentiated,fully active,wild-type,No,No,No,No,No,0


Process of Colorectal_201_alli
- master.csv process

In [4]:
df = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_201/master.csv')

df = df.rename(columns={
    'ANON_ID': 'id',
    'arm': 'arm',
    'SEX': 'sex',
    'abd_adh': 'abdominal wall adhesion',
    'bow_adh': 'bowel adhesion',
    'any_adh': 'any adhesion',
    'compall': 'any complications',
    'complica': 'surgical complications',
    'daysnarc': 'length of narcotic use',
    'diffgrp': 'histology',
    'factorb': 'pathology',
    'FU_STAT': 'target label'
}
)

df['sex'] = df['sex'].replace({'m':'male','f':'female', np.nan: 'male'})
df['abdominal wall adhesion'] = df['abdominal wall adhesion'].apply(lambda x: 'Yes' if x == 2 else 'No')
df['bowel adhesion'] = df['bowel adhesion'].apply(lambda x: 'Yes' if x == 2 else 'No')
df['any adhesion'] = df['any adhesion'].apply(lambda x: 'Yes' if x == 1 else 'No')
df['any complications'] = df['any complications'].apply(lambda x: 'Yes' if x == 1 else 'No')
df['surgical complications'] = df['surgical complications'].apply(lambda x: 'Yes' if x == 2 else 'No')
histo_map = {
    1:'well differentiated',
    2:'poorly differentiated'
}
freq_histo = df['histology'].map(histo_map).mode()[0]
df['histology'] = df['histology'].map(ecog_mappings)
df['histology'].fillna(value=freq_histo, inplace=True)
df['histology'].replace(to_replace=np.nan, value=freq_histo, inplace=True)
df['arm'] = df['arm'].replace({'A': 'conventional surgery', 'B': 'laparoscopic-assisted colectomy'})
df['length of narcotic use'] = df['length of narcotic use'].fillna(df['length of narcotic use'].median()).round()
df['target label'] = df['target label'].replace({1: 0, 2: 1})

df_processed_201 = df[[
    'id',
    'arm',
    'sex',
    'abdominal wall adhesion',
    'bowel adhesion',
    'any adhesion',
    'surgical complications',
    'any complications',
    'length of narcotic use',
    'histology',
    'pathology',
    'target label'
]]

# print(df_processed_201.isnull().sum())

df_processed_201.drop('id', axis=1).to_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_201/Colorectal_201.csv')

num_feat_list1 = ['length of narcotic use']
bin_feat_list1 = ['abdominal wall adhesion', 'bowel adhesion', 'bowel obstruction', 'any adhesion', 'surgical complications', 'any complications']

with open('/home/alexandra/Downloads/datasets/ae_again/Colorectal_201/numerical_feature.txt','w') as f:
    for x in num_feat_list1:
        f.write(x+'\n')
with open('/home/alexandra/Downloads/datasets/ae_again/Colorectal_201/binary_feature.txt','w') as f:
    for x in bin_feat_list1:
        f.write(x+'\n')

df_processed_201.head()

,id,arm,sex,abdominal wall adhesion,bowel adhesion,any adhesion,surgical complications,any complications,length of narcotic use,histology,pathology,target label
0,1,conventional surgery,female,No,No,No,No,No,2.0,symptoms but ambulatory,malignant,0
1,2,conventional surgery,male,Yes,No,Yes,No,No,3.0,symptoms but ambulatory,malignant,1
2,3,conventional surgery,female,No,No,No,No,Yes,2.0,symptoms but ambulatory,malignant,1
3,4,conventional surgery,female,No,No,No,No,No,4.0,symptoms but ambulatory,malignant,0
4,5,conventional surgery,female,No,No,No,No,No,3.0,symptoms but ambulatory,malignant,0


Process of Colorectal_309_amg
- sas -> csv

In [5]:
directory = '/home/alexandra/Downloads/datasets/ae_again/Colorectal_309'

for filename in os.listdir(directory):
    if filename.endswith('.sas7bdat'):
        filepath = os.path.join(directory, filename)
        df, meta = pyreadstat.read_sas7bdat(filepath)
        csv_filename = filename.replace('.sas7bdat', '.csv')
        csv_filepath = os.path.join(directory, csv_filename)
        df.to_csv(csv_filepath, index=False)
        print(f"Converted {filename} to CSV.")


- adsl_pds2019.csv process

In [20]:
df_sl = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_309/adsl_pds2019.csv')

df_sl = df_sl.rename(columns={
    'SUBJID': 'id',
    'AGE': 'age',
    'SEX': 'sex',
    'RACE': 'race',
    'B_WEIGHT': 'weight',
    'B_HEIGHT': 'height',
    'ATRT': 'arm',
    'B_ECOG': 'ECOG performance status',
    'DIAGTYPE': 'primary tumor diagnosis',
    'DTH': 'target label'
}
)

df_sl['age'] = df_sl['age'].astype(float)
df_sl['ECOG performance status'] = df_sl['ECOG performance status'].str.lower()
df_sl['sex'] = df_sl['sex'].str.lower()
df_sl['primary tumor diagnosis'] = df_sl['primary tumor diagnosis'].str.lower()
df_sl['arm'] = df_sl['arm']. replace({'FOLFOX alone': 'FOLFOX', 'Panitumumab + FOLFOX': 'FOLFOX + Panitumumab'})
df_sl['race'] = df_sl['race'].replace({'White or Caucasian': 'white', 
                                       'black or african american': 'black', 
                                       'Hispanic or Latino': 'other',
                                       'Asian': 'other',
                                       'Other': 'other'})
# weight + height -> bmi
df_sl['height'] = df_sl['height'] / 100
df_sl['bmi'] = df_sl['weight'] / (df_sl['height'] ** 2)
df_sl['bmi'] = df_sl['bmi'].replace(np.nan, df_sl['bmi'].median()).round(3)
df_sl = df_sl.drop(columns=['weight', 'height'])

df_SL = df_sl[[
    'id',
    'age',
    'sex',
    'race',
    'bmi',
    'arm',
    'ECOG performance status',
    'primary tumor diagnosis',
    'target label'
]]

# print(df_SL.isnull().sum())
df_SL.head()

,id,age,sex,race,bmi,arm,ECOG performance status,primary tumor diagnosis,target label
0,1,64.0,male,white,23.020,FOLFOX,symptoms but ambulatory,colon,1
1,3,65.0,male,white,24.610,FOLFOX + Panitumumab,symptoms but ambulatory,colon,1
2,4,60.0,male,white,24.490,FOLFOX,symptoms but ambulatory,colon,1
3,5,64.0,female,white,20.312,FOLFOX,symptoms but ambulatory,colon,1
4,6,74.0,male,white,22.571,FOLFOX,fully active,rectal,1


- adea_pds2019.csv process

In [7]:
df_adae = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_309/adae_pds2019.csv')

df_adae = df_adae.rename(columns={
    'SUBJID': 'id',
    'AEPT': 'tox',
    'AESEVCD': 'grade'
}
)
df_adae['is serious'] = df_adae['grade'].apply(lambda x: 1 if x > 2 else 0)
ae_list1 = df_adae['tox'].value_counts().index.tolist()
ae_name_list1 = []
for ae in ae_list1:
    ae_name1 = 'adverse effect: ' + ae.lower()
    df_adae[ae_name1] = np.zeros(len(df_adae))
    df_adae.loc[df_adae['tox']==ae,ae_name1] = 1
    df_adae[ae_name1] = df_adae[ae_name1] * df_adae['is serious']
    ae_name_list1.append(ae_name1)


/tmp/ipykernel_114933/2763952542.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_adae[ae_name1] = np.zeros(len(df_adae))
/tmp/ipykernel_114933/2763952542.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_adae[ae_name1] = np.zeros(len(df_adae))
/tmp/ipykernel_114933/2763952542.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmen

In [36]:
df_adae = df_adae[ae_name_list1 + ['id']].groupby('id').max().reset_index()
ae_cols = ['adverse effect: rash','adverse effect: dermatitis acneiform','adverse effect: acne']
df_slae = pd.merge(df_SL, df_adae[ae_cols+['id']], on='id', how='left').fillna(0)
df_slae[ae_cols]= df_slae[ae_cols].replace({0:'No',1:'Yes'})

df_slae.head()

,id,age,sex,race,bmi,arm,ECOG performance status,primary tumor diagnosis,target label,adverse effect: rash,adverse effect: dermatitis acneiform,adverse effect: acne
0,1,64.0,male,white,23.020,FOLFOX,symptoms but ambulatory,colon,1,No,No,No
1,3,65.0,male,white,24.610,FOLFOX + Panitumumab,symptoms but ambulatory,colon,1,No,No,No
2,4,60.0,male,white,24.490,FOLFOX,symptoms but ambulatory,colon,1,No,No,No
3,5,64.0,female,white,20.312,FOLFOX,symptoms but ambulatory,colon,1,No,No,No
4,6,74.0,male,white,22.571,FOLFOX,fully active,rectal,1,No,No,No


- biomark_pds2019.csv process

In [22]:
df_biomark = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_309/biomark_pds2019.csv')

df_biomark = df_biomark.rename(columns={
    'SUBJID': 'id',
    'BMMTR1': 'biomark KRAS',
}
)
df_biomark['biomark KRAS'] = df_biomark['biomark KRAS'].replace(['Failure', np.nan], 'indeterminate').str.lower()
df_bio = df_biomark[[
    'id',
    'biomark KRAS'
]]
df_processed = pd.merge(df_slae, df_bio, on='id', how='left')

df_processed_309 = df_processed[[
    'id',
    'arm',
    'sex',
    'age',
    'race',
    'bmi',
    'ECOG performance status',
    'primary tumor diagnosis',
    'biomark KRAS',
    'adverse effect: rash',
    'adverse effect: dermatitis acneiform',
    'adverse effect: acne',
    'target label'
]]

df_processed_309.drop('id', axis=1).to_csv('/home/alexandra/Downloads/datasets/ae_again/Colorectal_309/Colorectal_309.csv')

num_feat_list2 = ['age', 'bmi']
bin_feat_list2 = ['adverse effect: rash', 'adverse effect: dermatitis acneiform', 'adverse effect: acne']

with open('/home/alexandra/Downloads/datasets/ae_again/Colorectal_309/numerical_feature.txt','w') as f:
    for x in num_feat_list2:
        f.write(x+'\n')
with open('/home/alexandra/Downloads/datasets/ae_again/Colorectal_309/binary_feature.txt','w') as f:
    for x in bin_feat_list2:
        f.write(x+'\n')

# print(df_processed_309.isnull().sum())
df_processed_309.head()

,id,arm,sex,age,race,bmi,ECOG performance status,primary tumor diagnosis,biomark KRAS,adverse effect: rash,adverse effect: dermatitis acneiform,adverse effect: acne,target label
0,1,FOLFOX,male,64.0,white,23.020,symptoms but ambulatory,colon,indeterminate,No,No,No,1
1,3,FOLFOX + Panitumumab,male,65.0,white,24.610,symptoms but ambulatory,colon,mutant,No,No,No,1
2,4,FOLFOX,male,60.0,white,24.490,symptoms but ambulatory,colon,mutant,No,No,No,1
3,5,FOLFOX,female,64.0,white,20.312,symptoms but ambulatory,colon,mutant,No,No,No,1
4,6,FOLFOX,male,74.0,white,22.571,fully active,rectal,indeterminate,No,No,No,1


Process of Esophageal_213_alli
- NCT00052910_D1_(EFFICACY).csv

In [28]:
df_eff = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Esophagael_213/NCT00052910_D1_(EFFICACY).csv')

df_eff = df_eff.rename(columns={
    'MASK_ID': 'id',
    'TREAT_ASSIGNED': 'arm',
    'SEX_ID': 'sex',
    'RACE_ID': 'race',
    'agecat': 'age',
    'Hx_oth_cancer': 'any prior cancer',
    'prior_chemo': 'prior chemotheraphy',
    'prior_radiation': 'prior radiation',
    'PERFORMANCE_ID': 'ECOG performance status',
    'Histologic_grade': 'histology',
    'death_final': 'target_label'
}
)

def assign_random_age(age_cat):
    if age_cat == 1:
        # Age category 1: <60 years
        return np.random.randint(45, 60)
    elif age_cat == 2:
        # Age category 2: >=60 years
        return np.random.randint(60, 76)
    else:
        return None

df_eff['age'] = df_eff['age'].apply(assign_random_age).astype(float)
df_eff['race']= df_eff['race'].replace({1: 'white', 3: 'black', 4: 'other', 9: 'other'})
df_eff['sex']= df_eff['sex'].replace({1: 'male', 2: 'female'})
df_eff['arm']= df_eff['arm'].replace({1: '5FU/LV + Radiation', 2: 'ECF + Radiation'})
df_eff['any prior cancer'] = df_eff['any prior cancer'].apply(lambda x: 'Yes' if x == 2 else 'No')
df_eff['prior chemotheraphy'] = df_eff['prior chemotheraphy'].apply(lambda x: 'Yes' if x == 1 else 'No')
df_eff['prior radiation'] = df_eff['prior radiation'].apply(lambda x: 'Yes' if x == 1 else 'No')
df_eff['ECOG performance status'] = df_eff['ECOG performance status'].map(ecog_mappings)
histo_map = {
    1: 'well differentiated',
    2: 'well differentiated',
    3: 'poorly differentiated',
    4: 'poorly differentiated'
}
freq_his = df_eff['histology'].map(histo_map).mode()[0]
df_eff['histology'] = df_eff['histology'].map(histo_map)
df_eff['histology'].fillna(value=freq_his, inplace=True)
df_eff['histology'].replace(to_replace=np.nan, value=freq_his, inplace=True)

df_EFF = df_eff[[
    'id',
    'arm',
    'sex',
    'race',
    'age',
    'any prior cancer',
    'prior chemotheraphy',
    'prior radiation',
    'ECOG performance status',
    'histology',
    'target_label'
]]

# print(df_EFF.isnull().sum())
df_EFF.head()

,id,arm,sex,race,age,any prior cancer,prior chemotheraphy,prior radiation,ECOG performance status,histology,target_label
0,1,ECF + Radiation,male,white,57.0,No,No,No,symptoms but ambulatory,poorly differentiated,0
1,2,5FU/LV + Radiation,female,black,49.0,No,No,No,symptoms but ambulatory,poorly differentiated,0
2,3,5FU/LV + Radiation,male,white,53.0,No,No,No,fully active,poorly differentiated,1
3,4,5FU/LV + Radiation,female,white,59.0,No,No,No,symptoms but ambulatory,poorly differentiated,0
4,5,5FU/LV + Radiation,female,black,73.0,No,No,No,symptoms but ambulatory,poorly differentiated,1


- NCT00052910_D2_(AE).csv process

In [45]:
df_eae = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Esophagael_213/NCT00052910_D2_(AE).csv')

df_eae = df_eae.rename(columns={'MASK_ID': 'id'})
df_eae.dropna(subset=['EventName', 'GRADE_ID'], inplace=True)
df_eae['is serious'] = df_eae['GRADE_ID'].apply(lambda x: 1 if x > 2 else 0)
df_eae['EventName'] = df_eae['EventName'].str.replace(r'\s*\([^)]*\)', '', regex=True)
ae_list2 = df_eae['EventName'].value_counts().index.tolist()
ae_name_list2 = []

for ae in ae_list2:
    ae_name2 = 'adverse effect: ' + ae.lower()
    df_eae[ae_name2] = np.zeros(len(df_eae))
    df_eae.loc[df_eae['EventName'] == ae, ae_name2] = 1
    df_eae[ae_name2] *= df_eae['is serious']
    ae_name_list2.append(ae_name2)

/tmp/ipykernel_114933/2872924069.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_eae[ae_name2] = np.zeros(len(df_eae))
/tmp/ipykernel_114933/2872924069.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_eae[ae_name2] = np.zeros(len(df_eae))
/tmp/ipykernel_114933/2872924069.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented 

In [48]:
df_eae = df_eae[ae_name_list2 + ['id']].groupby('id').max().reset_index()
# sums = df_eae.sum()
# if 'MASK_ID' in sums:
#     sums = sums.drop('MASK_ID')
# sorted_sums = sums.sort_values(ascending=False)
# print(sorted_sums.head(6))

ae_sub_cols = ['adverse effect: neutrophils/granulocytes','adverse effect: leukocytes','adverse effect: nausea', 'adverse effect: anorexia', 'adverse effect: fatigue']
df_processed = pd.merge(df_EFF, df_eae[ae_sub_cols+['id']], on='id', how='left').fillna(0)
df_processed[ae_sub_cols]= df_processed[ae_sub_cols].replace({0:'No',1:'Yes'})

df_processed_213 = df_processed[[
    'id',
    'arm',
    'sex',
    'race',
    'age',
    'any prior cancer',
    'prior chemotheraphy',
    'prior radiation',
    'ECOG performance status',
    'histology',
    'adverse effect: neutrophils/granulocytes',
    'adverse effect: leukocytes',
    'adverse effect: nausea',
    'adverse effect: anorexia',
    'adverse effect: fatigue',
    'target_label'
]]

df_processed_213.drop('id', axis=1).to_csv('/home/alexandra/Downloads/datasets/ae_again/Esophagael_213/Esophagael_213.csv')

num_feat_list3 = ['age']
bin_feat_list3 = ['any prior cancer', 'prior chemotheraphy', 'prior radiation'] + ae_sub_cols

with open('/home/alexandra/Downloads/datasets/ae_again/Esophagael_213/numerical_feature.txt','w') as f:
    for x in num_feat_list3:
        f.write(x+'\n')
with open('/home/alexandra/Downloads/datasets/ae_again/Esophagael_213/binary_feature.txt','w') as f:
    for x in bin_feat_list3:
        f.write(x+'\n')

df_processed_213.head()

,id,arm,sex,race,age,any prior cancer,prior chemotheraphy,prior radiation,ECOG performance status,histology,adverse effect: neutrophils/granulocytes,adverse effect: leukocytes,adverse effect: nausea,adverse effect: anorexia,adverse effect: fatigue,target_label
0,1,ECF + Radiation,male,white,57.0,No,No,No,symptoms but ambulatory,poorly differentiated,No,No,No,No,No,0
1,2,5FU/LV + Radiation,female,black,49.0,No,No,No,symptoms but ambulatory,poorly differentiated,No,No,No,No,No,0
2,3,5FU/LV + Radiation,male,white,53.0,No,No,No,fully active,poorly differentiated,No,No,No,No,No,1
3,4,5FU/LV + Radiation,female,white,59.0,No,No,No,symptoms but ambulatory,poorly differentiated,No,No,No,No,No,0
4,5,5FU/LV + Radiation,female,black,73.0,No,No,No,symptoms but ambulatory,poorly differentiated,Yes,Yes,No,No,No,1


Process of Lung_261_alli
- sas -> csv

In [49]:
directory = '/home/alexandra/Downloads/datasets/ae_again/Lung_261'

for filename in os.listdir(directory):
    if filename.endswith('.sas7bdat'):
        filepath = os.path.join(directory, filename)
        df, meta = pyreadstat.read_sas7bdat(filepath)
        csv_filename = filename.replace('.sas7bdat', '.csv')
        csv_filepath = os.path.join(directory, csv_filename)
        df.to_csv(csv_filepath, index=False)
        print(f"Converted {filename} to CSV.")

Converted c9732_demographic.sas7bdat to CSV.
Converted c9732_ae.sas7bdat to CSV.
Converted c9732_fmt.sas7bdat to CSV.


- c9732_demographic.csv process

In [77]:
df_demog = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Lung_261/c9732_demographic.csv')

df_demog = df_demog.rename(columns={
    'PHATOM_ID': 'id',
    'TRT_ARM_LABEL': 'arm',
    'GENDER': 'sex',
    'RACE': 'race',
    'AGE': 'age',
    'CHEMO_CYCLE': 'number of chemotherapy cycles received', 
    'WGT_LOSSPCT': 'weight loss > 5/10%',
    'PS': 'ECOG performance status',
    'STATUS': 'target_label'
}
)
race_map = {
    1: 'other',
    2: 'other',
    3: 'black',
    5: 'white',
    99: 'other',
    np.nan: 'other'
}
df_demog['race'] = df_demog['race'].map(race_map)
df_demog['age'] = df_demog['age'].astype(float)
df_demog['sex']= df_demog['sex'].replace({1: 'male', 2: 'female'})
df_demog['weight loss > 5/10%'] = df_demog['weight loss > 5/10%'].apply(lambda x: 'Yes' if x == 2 else 'No')
df_demog['target_label'] = df_demog['target_label'].replace({1: 0, 2: 1, 3: 0})
df_demog['arm'] = df_demog['arm'].replace({
    'cisplatin,etoposide': 'Cisplatin + Etoposide',
    'paclitaxel,cisplatin,etoposide': 'Cisplatin + Etoposide + Paclitaxel' 
})
ecog_map = {
    0: 'fully active',
    1: 'symptoms but ambulatory',
    2: 'in bed less than 50% of the time',
    3: 'in bed less than 50% of the time',
    4: 'in bed less than 50% of the time',
    99: np.nan
}
freq_ps = df_demog['ECOG performance status'].map(ecog_map).mode()[0]
df_demog['ECOG performance status'] = df_demog['ECOG performance status'].map(ecog_map)
df_demog['ECOG performance status'].fillna(value=freq_ps, inplace=True)
df_demog['ECOG performance status'].replace(to_replace=np.nan, value=freq_ps, inplace=True)

df_demog['number of chemotherapy cycles received'] = df_demog['number of chemotherapy cycles received'].fillna(df_demog['number of chemotherapy cycles received'].median()).astype(float).round()

df_DEMOG = df_demog[[
    'id',
    'arm',
    'sex',
    'race',
    'age',
    'number of chemotherapy cycles received',
    'weight loss > 5/10%',
    'ECOG performance status',
    'target_label'
]]

# print(df_DEMOG.isnull().sum())
df_DEMOG.head()

,id,arm,sex,race,age,number of chemotherapy cycles received,weight loss > 5/10%,ECOG performance status,target_label
0,12727,Cisplatin + Etoposide,female,white,56.0,4.0,No,fully active,1
1,13700,Cisplatin + Etoposide + Paclitaxel,female,white,62.0,8.0,No,fully active,1
2,14768,Cisplatin + Etoposide,male,white,62.0,8.0,No,symptoms but ambulatory,1
3,14795,Cisplatin + Etoposide,male,white,41.0,8.0,Yes,symptoms but ambulatory,1
4,16718,Cisplatin + Etoposide + Paclitaxel,female,white,59.0,8.0,No,symptoms but ambulatory,1


- c9732_ae.csv process

In [61]:
df_lae = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Lung_261/c9732_ae.csv')

df_lae = df_lae.rename(columns={'PHATOM_ID': 'id'})
df_lae['is serious'] = df_lae['AE_GRADE'].apply(lambda x: 1 if x > 2 else 0)
df_lae['AE_NAME'] = df_lae['AE_NAME'].apply(lambda x: 'white blood cells' if x == 'wbc' else x)
ae_list3 = df_lae['AE_NAME'].value_counts().index.tolist()
ae_name_list3 = []

for ae in ae_list3:
    ae_name3 = 'adverse effect: ' + ae.lower()
    df_lae[ae_name3] = np.zeros(len(df_lae))
    df_lae.loc[df_lae['AE_NAME'] == ae, ae_name3] = 1
    df_lae[ae_name3] *= df_lae['is serious']
    ae_name_list3.append(ae_name3)

In [73]:
df_lae = df_lae[ae_name_list3 + ['id']].groupby('id').max().reset_index()
# sums = df_lae.sum()
# if 'id' in sums:
#     sums = sums.drop('id')
# sorted_sums = sums.sort_values(ascending=False)
# print(sorted_sums.head(10))

ae_sub_col = ['adverse effect: granulocytes/bands','adverse effect: white blood cells','adverse effect: lymphocytes', 'adverse effect: nausea']
df_processed = pd.merge(df_DEMOG, df_lae[ae_sub_col+['id']], on='id', how='left').fillna(0)
df_processed[ae_sub_col]= df_processed[ae_sub_col].replace({0:'No',1:'Yes'})

df_processed_261 = df_processed[[
    'id',
    'arm',
    'sex',
    'race',
    'age',
    'number of chemotherapy cycles received',
    'weight loss > 5/10%',
    'ECOG performance status',
    'adverse effect: granulocytes/bands',
    'adverse effect: white blood cells',
    'adverse effect: lymphocytes',
    'adverse effect: nausea',
    'target_label'
]]

df_processed_261.drop('id', axis=1).to_csv('/home/alexandra/Downloads/datasets/ae_again/Lung_261/Lung_261.csv')

num_feat_list4 = ['age', 'number of chemotherapy cycles received']
bin_feat_list4 = ['weight loss > 5/10%'] + ae_sub_col

with open('/home/alexandra/Downloads/datasets/ae_again/Lung_261/numerical_feature.txt','w') as f:
    for x in num_feat_list4:
        f.write(x+'\n')
with open('/home/alexandra/Downloads/datasets/ae_again/Lung_261/binary_feature.txt','w') as f:
    for x in bin_feat_list4:
        f.write(x+'\n')

df_processed_261.head()

,id,arm,sex,race,age,number of chemotherapy cycles received,weight loss > 5/10%,ECOG performance status,adverse effect: granulocytes/bands,adverse effect: white blood cells,adverse effect: lymphocytes,adverse effect: nausea,target_label
0,12727,Cisplatin + Etoposide,female,white,56.0,4.0,No,fully active,Yes,No,No,No,1
1,13700,Cisplatin + Etoposide + Paclitaxel,female,white,62.0,8.0,No,fully active,Yes,Yes,Yes,No,1
2,14768,Cisplatin + Etoposide,male,white,62.0,8.0,No,symptoms but ambulatory,Yes,Yes,Yes,No,1
3,14795,Cisplatin + Etoposide,male,white,41.0,8.0,Yes,symptoms but ambulatory,No,No,No,No,1
4,16718,Cisplatin + Etoposide + Paclitaxel,female,white,59.0,8.0,No,symptoms but ambulatory,Yes,Yes,Yes,No,1


Process of Lung_218_alli
- clinical.csv process

In [67]:
df_clin = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Lung_218/clinical.csv')

df_clin = df_clin.rename(columns={
    'mask_id': 'id',
    'arm': 'arm',
    'sex':'sex',
    'agecat': 'age',
    'race': 'race',
    'PS': 'ECOG performance status',
    'survcens':'target_label',
}
)
def assign_age(age_cat):
    if age_cat == 1:
        # Age category 1: <65 years
        return np.random.randint(45, 65)
    elif age_cat == 2:
        # Age category 2: >=65 years
        return np.random.randint(65, 75)
    else:
        return None

df_clin['age'] = df_clin['age'].apply(assign_age).astype(float)
df_clin['sex'] = df_clin['sex'].replace({1:'male', 2:'female'})
df_clin['race']= df_clin['race'].replace({1: 'white', 3: 'black'})
df_clin['arm'] = df_clin['arm'].replace({1: 'Sunitinib', 2: 'Placebo'})
df_clin['ECOG performance status'] = df_clin['ECOG performance status'].map(ecog_mappings)

df_CLIN = df_clin[[
    'id',
    'arm',
    'sex',
    'age',
    'race',
    'ECOG performance status',
    'target_label',
]]

# print(df_CLIN.isnull().sum())
df_CLIN.head()

,id,arm,sex,age,race,ECOG performance status,target_label
0,1,Placebo,female,66.0,white,in bed less than 50% of the time,1
1,2,Sunitinib,female,47.0,white,fully active,1
2,3,Sunitinib,female,71.0,white,symptoms but ambulatory,1
3,4,Sunitinib,female,64.0,white,fully active,1
4,5,Sunitinib,female,47.0,white,in bed less than 50% of the time,1


- ae.csv process

In [68]:
df_aee = pd.read_csv('/home/alexandra/Downloads/datasets/ae_again/Lung_218/ae.csv')

df_aee = df_aee.rename(columns={'mask_id': 'id'})
df_aee['is serious'] = df_aee['GRADE_ID'].apply(lambda x: 1 if x > 2 else 0)
df_aee['EventName'] = df_aee['EventName'].str.replace(r'\s*\([^)]*\)', '', regex=True)
ae_list4 = df_aee['EventName'].value_counts().index.tolist()
ae_name_list4 = []

for ae in ae_list4:
    ae_name4 = 'adverse effect: ' + ae.lower()
    df_aee[ae_name4] = np.zeros(len(df_aee))
    df_aee.loc[df_aee['EventName'] == ae, ae_name4] = 1
    df_aee[ae_name4] *= df_aee['is serious']
    ae_name_list4.append(ae_name4)

In [71]:
df_aee = df_aee[ae_name_list4 + ['id']].groupby('id').max().reset_index()
# sums = df_aee.sum()
# if 'id' in sums:
#     sums = sums.drop('id')
# sorted_sums = sums.sort_values(ascending=False)
# print(sorted_sums.head())

ae_sub_columns = ['adverse effect: fatigue','adverse effect: neutrophils/granulocytes','adverse effect: platelets', 'adverse effect: leukocytes']
df_processed = pd.merge(df_CLIN, df_aee[ae_sub_columns + ['id']], on='id', how='left').fillna(0)
df_processed[ae_sub_columns]= df_processed[ae_sub_columns].replace({0:'No',1:'Yes'})

df_processed_218 = df_processed[[
    'id',
    'arm',
    'sex',
    'race',
    'age',
    'ECOG performance status',
    'adverse effect: fatigue',
    'adverse effect: neutrophils/granulocytes',
    'adverse effect: platelets',
    'adverse effect: leukocytes',
    'target_label'
]]

df_processed_218.drop('id', axis=1).to_csv('/home/alexandra/Downloads/datasets/ae_again/Lung_218/Lung_218.csv')

num_feat_list5 = ['age']
bin_feat_list5 = ae_sub_columns

with open('/home/alexandra/Downloads/datasets/ae_again/Lung_218/numerical_feature.txt','w') as f:
    for x in num_feat_list5:
        f.write(x+'\n')
with open('/home/alexandra/Downloads/datasets/ae_again/Lung_218/binary_feature.txt','w') as f:
    for x in bin_feat_list5:
        f.write(x+'\n')

df_processed_218.head()

,id,arm,sex,race,age,ECOG performance status,adverse effect: fatigue,adverse effect: neutrophils/granulocytes,adverse effect: platelets,adverse effect: leukocytes,target_label
0,1,Placebo,female,white,66.0,in bed less than 50% of the time,No,No,No,No,1
1,2,Sunitinib,female,white,47.0,fully active,No,No,No,No,1
2,3,Sunitinib,female,white,71.0,symptoms but ambulatory,No,No,Yes,No,1
3,4,Sunitinib,female,white,64.0,fully active,No,No,No,No,1
4,5,Sunitinib,female,white,47.0,in bed less than 50% of the time,No,No,No,No,1
